# T1543.004 - Launch Daemon
Adversaries may create or modify Launch Daemons to execute malicious payloads as part of persistence. Launch Daemons are plist files used to interact with Launchd, the service management framework used by macOS. Launch Daemons require elevated privileges to install, are executed for every user on a system prior to login, and run in the background without the need for user interaction. During the macOS initialization startup, the launchd process loads the parameters for launch-on-demand system-level daemons from plist files found in <code>/System/Library/LaunchDaemons/</code> and <code>/Library/LaunchDaemons/</code>. Required Launch Daemons parameters include a <code>Label</code> to identify the task, <code>Program</code> to provide a path to the executable, and <code>RunAtLoad</code> to specify when the task is run. Launch Daemons are often used to provide access to shared resources, updates to software, or conduct automation tasks.(Citation: AppleDocs Launch Agent Daemons)(Citation: Methods of Mac Malware Persistence)(Citation: launchd Keywords for plists)

Adversaries may install a Launch Daemon configured to execute at startup by using the <code>RunAtLoad</code> parameter set to <code>true</code> and the <code>Program</code> parameter set to the malicious executable path. The daemon name may be disguised by using a name from a related operating system or benign software (i.e. [Masquerading](https://attack.mitre.org/techniques/T1036)). When the Launch Daemon is executed, the program inherits administrative permissions.(Citation: WireLurker)(Citation: OSX Malware Detection)

Additionally, system configuration changes (such as the installation of third party package managing software) may cause folders such as <code>usr/local/bin</code> to become globally writeable. So, it is possible for poor configurations to allow an adversary to modify executables referenced by current Launch Daemon's plist files.(Citation: LaunchDaemon Hijacking)(Citation: sentinelone macos persist Jun 2019)

## Atomic Tests

### Atomic Test #1 - Launch Daemon
Utilize LaunchDaemon to launch `Hello World`

**Supported Platforms:** macos

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: The shared library must exist on disk at specified location (#{path_malicious_plist})

##### Check Prereq Commands:
```bash
if [ -f $PathToAtomicsFolder/T1543.004/src/atomicredteam_T1543_004.plist ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```bash
echo "The plist file doesn't exist. Check the path and try again."; exit 1;

```

In [ ]:
Invoke-AtomicTest T1543.004 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo cp $PathToAtomicsFolder/T1543.004/src/atomicredteam_T1543_004.plist /Library/LaunchDaemons/com.atomicredteam.plist
sudo launchctl load -w /Library/LaunchDaemons/com.atomicredteam.plist
```

In [ ]:
Invoke-AtomicTest T1543.004 -TestNumbers 1

#### Cleanup: 
```bash
sudo launchctl unload /Library/LaunchDaemons/com.atomicredteam.plist
sudo rm /Library/LaunchDaemons/com.atomicredteam.plist
sudo rm /tmp/T1543_004_atomicredteam.txt
```

In [ ]:
Invoke-AtomicTest T1543.004 -TestNumbers 1 -Cleanup

## Detection
Monitor for new files added to the <code>/Library/LaunchDaemons/</code> folder. The System LaunchDaemons are protected by SIP.

Some legitimate LaunchDaemons point to unsigned code that could be exploited. For Launch Daemons with the <code>RunAtLoad</code> parameter set to true, ensure the <code>Program</code> parameter points to signed code or executables are in alignment with enterprise policy. Some parameters are interchangeable with others, such as <code>Program</code> and <code>ProgramArguments</code> parameters but one must be present.(Citation: launchd Keywords for plists)

